# Look at the overview scans
Let's see what we did there.

In [1]:
import platform
import glob
import os
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib_scalebar.scalebar import ScaleBar
import matplotlib.patches
import seaborn
import pandas
import numpy
from tqdm import notebook
from itkwidgets import view  # 3d viewer
import imageio
import dask
import dask.array as da
import dask_image.imread
from distributed import Client
client = Client()
from numcodecs import Blosc

In [2]:
print('You can seee what DASK is doing at "http://localhost:%s/status"' % client.scheduler_info()['services']['dashboard'])

You can seee what DASK is doing at "http://localhost:8787/status"


In [3]:
# # Ignore warnings in the notebook
# import warnings
# warnings.filterwarnings("ignore")

In [4]:
# Set up figure defaults
plt.rc('image', cmap='gray', interpolation='nearest')  # Display all images in b&w and with 'nearest' interpolation
plt.rcParams['figure.figsize'] = (16, 9)  # Size up figures a bit
plt.rcParams['figure.dpi'] = 200  # Increase dpi

In [5]:
# Setup scale bar defaults
plt.rcParams['scalebar.location'] = 'lower right'
plt.rcParams['scalebar.frameon'] = False
plt.rcParams['scalebar.color'] = 'white'


The validate_legend_loc function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.


In [6]:
# Display all plots identically
lines = 4
# And then do something like
# plt.subplot(lines, numpy.ceil(len(Data) / float(lines)), c + 1)

In [7]:
# Different locations if running either on Linux or Windows
if 'anaklin25' in platform.node():
    FastSSD = True
else:
    FastSSD = False
# to speed things up significantly
if 'Linux' in platform.system():
    if FastSSD:
        BasePath = os.path.join(os.sep, 'media', 'habi', 'Fast_SSD')
    else:
        BasePath = os.path.join(os.sep, 'home', 'habi', '1272')
elif 'Darwin' in platform.system():
    BasePath = os.path.join('/Volumes/2TBSSD/')
else:
    if FastSSD:
        BasePath = os.path.join('F:\\')
    else:
        if 'anaklin' in platform.node():
            BasePath = os.path.join('S:\\')
        else:
            BasePath = os.path.join('D:\\Results')
Root = os.path.join(BasePath, 'Hearts Melly')
print('We are loading all the data from %s' % Root)

We are loading all the data from /Volumes/2TBSSD/Hearts Melly


In [8]:
glob.glob(os.path.join(Root))

['/Volumes/2TBSSD/Hearts\xa0Melly']

In [9]:
def get_pixelsize(logfile):
    """Get the pixel size from the scan log file"""
    with open(logfile, 'r') as f:
        for line in f:
            if 'Image Pixel' in line and 'Scaled' not in line:
                pixelsize = float(line.split('=')[1])
    return(pixelsize)

In [10]:
def get_filter(logfile):
    """Get the filter we used whole scanning from the scan log file"""
    with open(logfile, 'r') as f:
        for line in f:
            if 'Filter=' in line:
                whichfilter = line.split('=')[1].strip()
    return(whichfilter)

In [11]:
def get_exposuretime(logfile):
    """Get the exposure time size from the scan log file"""
    with open(logfile, 'r') as f:
        for line in f:
            if 'Exposure' in line:
                exposuretime = int(line.split('=')[1])
    return(exposuretime)

In [12]:
def get_ringartefact(logfile):
    """Get the ring artefact correction from the  scan log file"""
    with open(logfile, 'r') as f:
        for line in f:
            if 'Ring Artifact' in line:
                ringartefactcorrection = int(line.split('=')[1])
    return(ringartefactcorrection)

In [13]:
def get_beamhardening(logfile):
    """Get the beamhardening correction from the  scan log file"""
    with open(logfile, 'r') as f:
        for line in f:
            if 'Hardening' in line:
                beamhardeningcorrection = int(line.split('=')[1])
    return(beamhardeningcorrection)

In [14]:
def get_reconstruction_grayvalue(logfile):
    """How did we map the brightness of the reconstructions?"""
    with open(logfile, 'r') as f:
        grayvalue = numpy.nan
        for line in f:
            if 'Maximum for' in line:
                grayvalue = float(line.split('=')[1])
    return(grayvalue)

In [15]:
# Make us a dataframe for saving all that we need
Data = pandas.DataFrame()

In [16]:
# Get all folders
Data['Folder'] = glob.glob(os.path.join(Root, 'Rat[678T]*', '*'))

In [17]:
# Get rid of all non-folders
for c, i in Data.iterrows():
    if not os.path.isdir(i.Folder):
        Data.drop([c], inplace=True)
Data.reset_index(drop=True, inplace=True)

In [18]:
# Drop all folders we don't need
for c, row in Data.iterrows():
    if 'Rat' not in row.Folder:
        Data.drop([c], inplace=True)
    elif 'overview' not in row.Folder:
        Data.drop([c], inplace=True)
    elif 'Test' in row.Folder:
        Data.drop([c], inplace=True)
# Reset dataframe to something that we would get if we only would have loaded the 'rec' files
Data = Data.reset_index(drop=True)

In [19]:
Data['Sample'] = [os.path.basename(os.path.dirname(f)) for f in Data['Folder']]
Data['Scan'] = [os.path.basename(f) for f in Data['Folder']]
Data['SampleNameLength'] = [len(sn) for sn in Data['Sample']]
Data['ScanNameLength'] = [len(sn) for sn in Data['Scan']]

In [20]:
print('We habe %s scans to work with' % len(Data))

We habe 18 scans to work with


In [21]:
# Get log files of all the scans
Data['LogFile'] = [sorted(glob.glob(os.path.join(f, 'proj', '*.log')))[0] for f in Data['Folder']]

In [22]:
# Get parameters to doublecheck from logfiles
Data['Voxelsize'] = [get_pixelsize(log) for log in Data['LogFile']]
Data['Filter'] = [get_filter(log) for log in Data['LogFile']]
Data['Exposuretime'] = [get_exposuretime(log) for log in Data['LogFile']]
Data['Grayvalue'] = [get_reconstruction_grayvalue(log) for log in Data['LogFile']]
Data['RingartefactCorrection'] = [get_ringartefact(log) for log in Data['LogFile']]
Data['BeamHardeningCorrection'] = [get_beamhardening(log) for log in Data['LogFile']]

In [23]:
# Did we reconstruct them equally?
Data[['Sample', 'Voxelsize', 'Filter', 'Exposuretime', 'Grayvalue', 'RingartefactCorrection', 'BeamHardeningCorrection']]

,Sample,Voxelsize,Filter,Exposuretime,Grayvalue,RingartefactCorrection,BeamHardeningCorrection
0,Rat69,10.000036,Al 1mm,2349,0.07,14,0
1,Rat61,10.000036,Al 1mm,2349,0.07,14,0
2,Rat62,10.000036,Al 1mm,2349,0.07,14,0
3,Rat63,10.000036,Al 1mm,2349,0.07,14,0
4,Rat64,10.000036,Al 1mm,2349,0.07,14,0
5,Rat65,10.000036,Al 1mm,2349,0.07,14,0
6,Rat66,10.000036,Al 1mm,2349,0.07,14,0
7,Rat67,10.000036,Al 1mm,1593,0.07,14,0
8,Rat68,10.000036,Al 1mm,2349,0.07,14,0
9,Rat79,10.000036,Al 1mm,2330,0.07,14,0


In [24]:
print(Data['Voxelsize'].unique())
print(Data['Exposuretime'].unique())
print(Data['Filter'].unique())
print(Data['Grayvalue'].unique())
print(Data['RingartefactCorrection'].unique())
print(Data['BeamHardeningCorrection'].unique())

[10.000036]
[2349 1593 2330]
['Al 1mm']
[0.07]
[14]
[0]


In [25]:
# Save to XLS sheet for Ludovic
Data[['Folder', 'Sample', 'Scan', 'Voxelsize']].to_excel('Voxelsizes.xlsx')

In [26]:
# Get the file names of the reconstructions
Data['Reconstructions'] = [sorted(glob.glob(os.path.join(f,
                                                         'rec',
                                                         '*.png'))) for f in Data['Folder']]
Data['Number of reconstructions'] = [len(r) for r in Data.Reconstructions]

In [27]:
Data

,Folder,Sample,Scan,SampleNameLength,ScanNameLength,LogFile,Voxelsize,Filter,Exposuretime,Grayvalue,RingartefactCorrection,BeamHardeningCorrection,Reconstructions,Number of reconstructions
0,/Volumes/2TBSSD/Hearts Melly/Rat69/overview,Rat69,overview,5,8,/Volumes/2TBSSD/Hearts Melly/Rat69/overview/pr...,10.000036,Al 1mm,2349,0.07,14,0,[/Volumes/2TBSSD/Hearts Melly/Rat69/overview/r...,2213
1,/Volumes/2TBSSD/Hearts Melly/Rat61/overview,Rat61,overview,5,8,/Volumes/2TBSSD/Hearts Melly/Rat61/overview/pr...,10.000036,Al 1mm,2349,0.07,14,0,[/Volumes/2TBSSD/Hearts Melly/Rat61/overview/r...,2622
2,/Volumes/2TBSSD/Hearts Melly/Rat62/overview,Rat62,overview,5,8,/Volumes/2TBSSD/Hearts Melly/Rat62/overview/pr...,10.000036,Al 1mm,2349,0.07,14,0,[/Volumes/2TBSSD/Hearts Melly/Rat62/overview/r...,2853
3,/Volumes/2TBSSD/Hearts Melly/Rat63/overview,Rat63,overview,5,8,/Volumes/2TBSSD/Hearts Melly/Rat63/overview/pr...,10.000036,Al 1mm,2349,0.07,14,0,[/Volumes/2TBSSD/Hearts Melly/Rat63/overview/r...,2284
4,/Volumes/2TBSSD/Hearts Melly/Rat64/overview,Rat64,overview,5,8,/Volumes/2TBSSD/Hearts Melly/Rat64/overview/pr...,10.000036,Al 1mm,2349,0.07,14,0,[/Volumes/2TBSSD/Hearts Melly/Rat64/overview/r...,2401
5,/Volumes/2TBSSD/Hearts Melly/Rat65/overview,Rat65,overview,5,8,/Volumes/2TBSSD/Hearts Melly/Rat65/overview/pr...,10.000036,Al 1mm,2349,0.07,14,0,[/Volumes/2TBSSD/Hearts Melly/Rat65/overview/r...,2313
6,/Volumes/2TBSSD/Hearts Melly/Rat66/overview,Rat66,overview,5,8,/Volumes/2TBSSD/Hearts Melly/Rat66/overview/pr...,10.000036,Al 1mm,2349,0.07,14,0,[/Volumes/2TBSSD/Hearts Melly/Rat66/overview/r...,2429
7,/Volumes/2TBSSD/Hearts Melly/Rat67/overview,Rat67,overview,5,8,/Volumes/2TBSSD/Hearts Melly/Rat67/overview/pr...,10.000036,Al 1mm,1593,0.07,14,0,[/Volumes/2TBSSD/Hearts Melly/Rat67/overview/r...,2147
8,/Volumes/2TBSSD/Hearts Melly/Rat68/overview,Rat68,overview,5,8,/Volumes/2TBSSD/Hearts Melly/Rat68/overview/pr...,10.000036,Al 1mm,2349,0.07,14,0,[/Volumes/2TBSSD/Hearts Melly/Rat68/overview/r...,2533
9,/Volumes/2TBSSD/Hearts Melly/Rat79/overview,Rat79,overview,5,8,/Volumes/2TBSSD/Hearts Melly/Rat79/overview/pr...,10.000036,Al 1mm,2330,0.07,14,0,[],0


In [28]:
# Drop samples which have not been reconstructed yet
# Based on https://stackoverflow.com/a/13851602
for sample in Data[Data['Number of reconstructions'] == 0]['Sample']:
    print('%s has not been reconstructed yet, we remove it from our data temporarily' % sample)
    Data = Data[Data['Number of reconstructions'] > 0]
    Data.reset_index(drop=True, inplace=True)
print('We have %s folders with reconstructions' % (len(Data)))

Rat79 has not been reconstructed yet, we remove it from our data temporarily
Rat80 has not been reconstructed yet, we remove it from our data temporarily
Rat80b has not been reconstructed yet, we remove it from our data temporarily
Rat81 has not been reconstructed yet, we remove it from our data temporarily
Rat82 has not been reconstructed yet, we remove it from our data temporarily
Rat83 has not been reconstructed yet, we remove it from our data temporarily
Rat83b has not been reconstructed yet, we remove it from our data temporarily
Rat84 has not been reconstructed yet, we remove it from our data temporarily
We have 10 folders with reconstructions


In [29]:
# Display some info
for c, row in Data.iterrows():
    print('%02s/%s: %s has %03s reconstructions in %s/%s' % (c + 1,
                                                             len(Data),
                                                             row['Sample'].rjust(Data['SampleNameLength'].max()),
                                                             row['Number of reconstructions'],
                                                             os.path.basename(row['Folder']),
                                                             'rec'))

 1/10: Rat69 has 2213 reconstructions in overview/rec
 2/10: Rat61 has 2622 reconstructions in overview/rec
 3/10: Rat62 has 2853 reconstructions in overview/rec
 4/10: Rat63 has 2284 reconstructions in overview/rec
 5/10: Rat64 has 2401 reconstructions in overview/rec
 6/10: Rat65 has 2313 reconstructions in overview/rec
 7/10: Rat66 has 2429 reconstructions in overview/rec
 8/10: Rat67 has 2147 reconstructions in overview/rec
 9/10: Rat68 has 2533 reconstructions in overview/rec
10/10: Rat60 has 2795 reconstructions in overview/rec


In [31]:
asdfasdf==

SyntaxError: invalid syntax (<ipython-input-31-2eec9e9b1992>, line 1)

In [ ]:
# Convert reconstructions into a DASK array and save them to disk
# Partially based on http://stackoverflow.com/a/39195332/323100
# and on /LungMetastasis/HighResolutionScanAnalysis.ipynb
Data['OutputNameRec'] = [os.path.join(f, sample + '_rec.zarr') for f, sample in zip(Data.Folder, Data.Sample)]
Reconstructions = [numpy.nan for file in Data['OutputNameRec']]
for c, row in notebook.tqdm(Data.iterrows(),
                            desc='Converting reconstructions to .zarr files',
                            total=len(Data)):
    if not os.path.exists(row['OutputNameRec']):
        print('%2s/%s: Reading %s reconstructions and saving to %s' % (c + 1,
                                                                       len(Data),
                                                                       row['Number of reconstructions'],
                                                                       row['OutputNameRec'][len(Root):]))
        Reconstructions = dask_image.imread.imread(os.path.join(row['Folder'], 'rec', '*.png'))
        Reconstructions.rechunk(200).to_zarr(row['OutputNameRec'],
                                             overwrite=True,
                                             compressor=Blosc(cname='zstd',
                                                              clevel=5,
                                                              shuffle=Blosc.BITSHUFFLE))

In [ ]:
# Load the reconstructions a zarr arrays
Reconstructions = [dask.array.from_zarr(file) for file in Data['OutputNameRec']]

In [ ]:
# The three cardinal directions
directions = ['Axial', 'Sagittal', 'Coronal']

In [ ]:
def cropper(image,
            despeckle=True,
            verbose=False,
            threshold=50):
    import skimage.morphology
    import scipy
    '''Crop array to biggest item in it'''
    dimensions = len(image.shape)
    if verbose:
        print('Cropping %s-dimensional image' % dimensions)
    # Threshold
    thresholded = image > threshold
    if despeckle:
        if verbose:
            print('Removing small objects')
        despeckled = skimage.util.apply_parallel(skimage.morphology.remove_small_objects,
                                                 thresholded,
                                                 extra_keywords={'min_size': 10**dimensions},
                                                 chunks=[15,15,15])
    # Find biggest object
    # https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.find_objects.html
    if verbose:
        print('Finding objects')
    if despeckle:
        cropdimensions = scipy.ndimage.find_objects(despeckled)[0]
    else:
        cropdimensions = scipy.ndimage.find_objects(thresholded)[0]
    if verbose:
        print('Cutting the image down to')
        for c, cd in enumerate(cropdimensions):
            print('\t- %s: %s' % (c, cd))
        if dimensions > 2:
            # Calculate cardinal direction MIPs
            MIPs = [image.max(axis=d) for d, direction in notebook.tqdm(enumerate(directions),
                                                                                  desc='Calculating MIPs',
                                                                                  total=len(directions))]
            for d, direction in enumerate(directions):
                plt.subplot(1, dimensions, d + 1)
                plt.imshow(MIPs[d], alpha=0.618)
                plt.imshow(dask.array.ma.masked_less(MIPs[d]>threshold, 1),
                           alpha=0.309,
                           cmap='viridis_r')
                if d == 0:
                    plt.axhline(cropdimensions[1].start,
                                c=seaborn.color_palette()[0],
                                label=cropdimensions[1].start)
                    plt.axhline(cropdimensions[1].stop,
                                c=seaborn.color_palette()[1],
                                label=cropdimensions[1].stop)
                    plt.axvline(cropdimensions[2].start,
                                c=seaborn.color_palette()[2],
                                label=cropdimensions[2].start)
                    plt.axvline(cropdimensions[2].stop,
                                c=seaborn.color_palette()[3],
                                label=cropdimensions[2].stop)
                elif d == 1:
                    plt.axhline(cropdimensions[0].start,
                                c=seaborn.color_palette()[0],
                                label=cropdimensions[0].start)
                    plt.axhline(cropdimensions[0].stop,
                                c=seaborn.color_palette()[1],
                                label=cropdimensions[0].stop)
                    plt.axvline(cropdimensions[2].start,
                                c=seaborn.color_palette()[2],
                                label=cropdimensions[2].start)
                    plt.axvline(cropdimensions[2].stop,
                                c=seaborn.color_palette()[3],
                                label=cropdimensions[2].stop)                    
                elif d == 2:
                    plt.axhline(cropdimensions[0].start,
                                c=seaborn.color_palette()[0],
                                label=cropdimensions[0].start)
                    plt.axhline(cropdimensions[0].stop,
                                c=seaborn.color_palette()[1],
                                label=cropdimensions[0].stop)
                    plt.axvline(cropdimensions[1].start,
                                c=seaborn.color_palette()[2],
                                label=cropdimensions[1].start)
                    plt.axvline(cropdimensions[1].stop,
                                c=seaborn.color_palette()[3],
                                label=cropdimensions[1].stop)                                        
                plt.title('%s MIP' % direction)
                plt.legend(loc='lower right')
        else:
            plt.subplot(121)
            plt.imshow(image, alpha=0.618)
            plt.imshow(dask.array.ma.masked_less(thresholded, 1), alpha=0.618, cmap='viridis_r')
            plt.axhline(cropdimensions[0].start, c=seaborn.color_palette()[0], label=cropdimensions[0].start)
            plt.axhline(cropdimensions[0].stop, c=seaborn.color_palette()[1], label=cropdimensions[0].stop)
            plt.axvline(cropdimensions[1].start, c=seaborn.color_palette()[2], label=cropdimensions[1].start)
            plt.axvline(cropdimensions[1].stop, c=seaborn.color_palette()[3], label=cropdimensions[1].stop)
            plt.title('Original image with thresholded overlay')
            plt.legend(loc='lower right')
            plt.subplot(122)
            plt.imshow(image[cropdimensions])
            plt.title('Output')
        plt.show()
    if verbose:
        print('Cropped image from %s to %s' % (image.shape, image[cropdimensions].shape))
        print(cropdimensions)
    return(image[cropdimensions])

In [ ]:
# # Test cropping function
# img = Reconstructions[-1]
# Cropped = cropper(img, verbose=True, despeckle=True, threshold=55)
# for c, direction in enumerate(directions):
#     plt.subplot(1,3,c+1)    
#     if 'Axial' in direction:
#         plt.imshow(Cropped[Cropped.shape[0] // 2])
#     if 'Sagittal' in direction:
#         plt.imshow(Cropped[:, Cropped.shape[1] // 2, :])
#     if 'Coronal' in direction:
#         plt.imshow(Cropped[:, :, Cropped.shape[2] // 2])

In [ ]:
# cropper(Reconstructions[3], verbose=True)

In [ ]:
# Crop the reconstructions and save them out as .zarr files, too
Data['OutputNameRecCrop'] = [os.path.join(f, sample + '_rec_crop.zarr') for f, sample in zip(Data.Folder, Data.Sample)]
for c, row in notebook.tqdm(Data.iterrows(),
                            desc='Saving cropped samples to .zarr files',
                            total=len(Data)):
    if not os.path.exists(row['OutputNameRecCrop']):
        print('%3s/%3s: Saving to %s' % (c + 1,
                                         len(Data),
                                         row['OutputNameRecCrop'][len(Root):]))
        # Calculate crop with our function and directly write to ZARR file
        cropper(Reconstructions[c],
                verbose=False,
                despeckle=True).to_zarr(row['OutputNameRecCrop'],
                                        overwrite=True,
                                        compressor=Blosc(cname='zstd',
                                                         clevel=3,
                                                         shuffle=Blosc.BITSHUFFLE))  

In [ ]:
# Load the cropped zarr arrays as reconstructions
Reconstructions = [dask.array.from_zarr(file) for file in Data['OutputNameRecCrop']]

In [ ]:
# Save out cropped rec slices
for c, row in notebook.tqdm(Data.iterrows(),
                            desc='Saving out cropped recs',
                            total=len(Data)):
    os.makedirs(os.path.join(row.Folder, 'rec_crop'),
                exist_ok=True)
    for d, rec in notebook.tqdm(enumerate(Reconstructions[c]),
                                total=len(Reconstructions[c]),
                                desc=os.path.join(row.Sample, os.path.basename(row.Folder)),
                                leave=False):
        filename = os.path.join(row.Folder,
                                'rec_crop', str(row.Sample) + '_rec_crop_%08d.png' % d)
        if not os.path.exists(filename):
            imageio.imsave(filename, rec)

In [ ]:
# How big are the datasets?
Data['Size'] = [numpy.shape(rec) for rec in Reconstructions]

In [ ]:
# Data[['Sample','Size']]

In [ ]:
# Read or calculate the middle slices, put them into the dataframe and save them to disk
for d, direction in enumerate(directions):
    Data['Mid_' + direction] = [None] * len(Reconstructions)
for c, row in notebook.tqdm(Data.iterrows(), desc='Middle images', total=len(Data)):
    for d, direction in notebook.tqdm(enumerate(directions),
                                      desc=row['Sample'],
                                      leave=False,
                                      total=len(directions)):
        outfilepath = os.path.join(row['Folder'], '%s.Middle.%s.png' % (row['Sample'], direction))
        if os.path.exists(outfilepath):
            Data.at[c,'Mid_' + direction] = imageio.imread(outfilepath)
        else:
            # Generate requested axial view
            if 'Axial' in direction:
                Data.at[c,'Mid_' + direction] = Reconstructions[c][Data['Size'][c][0]//2].compute()
            if 'Sagittal' in direction:
                Data.at[c,'Mid_' + direction] = Reconstructions[c][:,Data['Size'][c][1]//2,:].compute()
            if 'Coronal' in direction:
                Data.at[c,'Mid_' + direction] = Reconstructions[c][:,:,Data['Size'][c][2]//2].compute()
            # Save the calculated 'direction' view to disk
            imageio.imwrite(outfilepath, (Data.at[c,'Mid_' + direction]))

In [ ]:
# Show middle slices
for c, row in Data.iterrows():
    outputfile = os.path.join(row['Folder'], row['Sample'] + '.MiddleSlices.png')
    for d, direction in enumerate(directions):
        plt.subplot(1, 3, d + 1)
        plt.imshow(row['Mid_' + direction])
        plt.gca().add_artist(ScaleBar(row['Voxelsize'], 'um'))
        plt.title('%s, %s, %s' % (row['Sample'],
                                  os.path.basename(row['Folder']),
                                  direction + ' Middle slice'))
        plt.axis('off')
    if not os.path.exists(outputfile):
        plt.savefig(outputfile,
                    transparent=True,
                    bbox_inches='tight')
    plt.show()

In [ ]:
# Read or calculate the directional MIPs, put them into the dataframe and save them to disk
for d, direction in enumerate(directions):
    Data['MIP_' + direction] = [None] * len(Reconstructions)
for c, row in notebook.tqdm(Data.iterrows(), desc='MIPs', total=len(Data)):
    for d, direction in notebook.tqdm(enumerate(directions),
                                      desc=row['Sample'],
                                      leave=False,
                                      total=len(directions)):
        outfilepath = os.path.join(row['Folder'], '%s.MIP.%s.png' % (row['Sample'], direction))
        if os.path.exists(outfilepath):
            Data.at[c,'MIP_' + direction] = imageio.imread(outfilepath)
        else:
            # Generate MIP
            Data.at[c,'MIP_' + direction] = Reconstructions[c].max(axis=d).compute()
            # Save it out
            imageio.imwrite(outfilepath, Data.at[c,'MIP_' + direction].astype('uint8'))

In [ ]:
# Show MIP slices
for c, row in Data.iterrows():
    outputfile = os.path.join(row['Folder'], row['Sample'] + '.MIPs.png')
    for d, direction in enumerate(directions):
        plt.subplot(1, 3, d + 1)
        plt.imshow(row['MIP_' + direction])
        plt.gca().add_artist(ScaleBar(row['Voxelsize'], 'um'))
        plt.title('%s, %s, %s' % (row['Sample']
                                  os.path.basename(row['Folder']),
                                  direction + ' Middle slice'))
        plt.axis('off')
    if not os.path.exists(outputfile):
        plt.savefig(outputfile,
                    transparent=True,
                    bbox_inches='tight')
    plt.show()

In [ ]:
#for c, row in Data.iterrows():
#    for d, direction in enumerate(directions):
#        print(row.Sample, direction, numpy.mean(row['Mid_' + direction]))

In [ ]:
# Show middle slices together
for d, direction in enumerate(directions):
    for c, row in Data.iterrows():
        plt.subplot(lines, numpy.ceil(len(Data) / float(lines)), c + 1)
        plt.imshow(row['Mid_' + direction])
        plt.gca().add_artist(ScaleBar(row['Voxelsize'], 'um'))
        if 'xial' in direction:
            recname = os.path.basename(row['Reconstructions'][row['Number of reconstructions'] // 2]).replace('0000',
                                                                                                              '')
            plt.title('%s\n%s' % (os.path.basename(row['Folder']), recname))
        else:
            plt.title('%s\n%s' % (row['Sample'], os.path.basename(row['Folder'])))
        plt.axis('off')
    plt.suptitle('Mid-' + direction + ' slice')
    plt.savefig(os.path.join(Root, 'MiddleSlices.' + direction + '.png'),
                transparent=True,               
                bbox_inches='tight')
    plt.show()

In [ ]:
# Show MIPs together
for d, direction in enumerate(directions):
    for c, row in Data.iterrows():
        plt.subplot(lines, numpy.ceil(len(Data) / float(lines)), c + 1)
        plt.imshow(row['MIP_' + direction])
        plt.gca().add_artist(ScaleBar(row['Voxelsize'], 'um'))
        plt.title('%s, %s' % (row['Sample'], os.path.basename(row['Folder'])))
        plt.axis('off')
        plt.suptitle(direction + ' MIPs')
    plt.savefig(os.path.join(Root, 'MIPs.' + direction + '.png'),
                transparent=True,
                bbox_inches='tight')
    plt.show()

In [ ]:
# Save mean of reconstruction gray values, which we can use for getting an overview of the image data
#Data['GrayValueMean'] = [rec.mean().compute() for rec in Reconstructions]

In [ ]:
# Save STD of reconstruction gray values, which we can use for getting an overview of the image data
#Data['GrayValueSTD'] = [rec.std().compute() for rec in Reconstructions]

In [ ]:
# Save out the dataframe to disk, so we can use it later in another notebook
#Data.to_pickle(os.path.join(Root, 'MellyDataframe.pkl'))

In [ ]:
#view(Reconstructions[2],
#     annotations=False)

In [ ]:
print('Done')